## Load the required csv 

In [2]:
import pandas as pd

inventories_fy23_path = "inventories_full_fy23.csv"
inventories_fy24_path = "inventories_full_fy24.csv"

inventories_fy23 = pd.read_csv(inventories_fy23_path)
inventories_fy24 = pd.read_csv(inventories_fy24_path)

In [3]:
inventories_fy23.head()

L1                             L2                            L3  \
0  Inventories                            NaN                           NaN   
1          NaN  Raw Material & Mfg. Componets                           NaN   
2          NaN                            NaN  Raw Material & Mfg. Componet   
3          NaN                            NaN                           NaN   
4          NaN                            NaN                           NaN   

                             L4          001-AS         133-FES  \
0                           NaN  53671516381.66  22944652152.91   
1                           NaN  36468349988.25    8158078316.8   
2                           NaN  33001594818.97   7779327668.78   
3     Closing Stock- Gunny Bags               0               0   
4  Clg Stock - Raw Material-B/s  30235079480.32   6542914168.47   

      008-SWARAJ 002-HO        Powerol CE  ... 004-MDS Synergy       007-SBU  \
0  9009308396.48      0  1325324816.29  0  ...       0       0  101756237.54   
1  1466552014.35      0   864491092.42  0  ...       0       0             0   
2     1309178305      0   864491092.42  0  ...       0       0             0   
3              0      0              0  0  ...       0       0             0   
4     1270706601      0   294446899.72  0  ...       0       0             0   

              TW 134-TPDS 135-INTNL OPER 138-CONT SOUR 140-NON MRV  \
0  1760985713.37        0              0             0           0   
1  1464491878.11        0              0             0           0   
2  1464491878.11        0              0             0           0   
3              0        0              0             0           0   
4  1464491878.11        0              0             0           0   

  141-SOURCING 139-CORP  
0            0        0  
1            0        0  
2            0        0  
3            0        0  
4            0        0  

[5 rows x 21 columns]

In [4]:
inventories_fy24.head()

L1                             L2                            L3  \
0  Inventories                            NaN                           NaN   
1          NaN  Raw Material & Mfg. Componets                           NaN   
2          NaN                            NaN  Raw Material & Mfg. Componet   
3          NaN                            NaN                           NaN   
4          NaN                            NaN                           NaN   

                             L4        001-AS       133-FES    008-SWARAJ  \
0                           NaN  6.398000e+10  1.979503e+10  8.952039e+09   
1                           NaN  2.929258e+10  7.299429e+09  1.312377e+09   
2                           NaN  2.763917e+10  6.952002e+09  1.140243e+09   
3     Closing Stock- Gunny Bags  0.000000e+00  0.000000e+00  0.000000e+00   
4  Clg Stock - Raw Material-B/s  2.353945e+10  6.113279e+09  1.096696e+09   

   002-HO       Powerol  CE  ...  004-MDS  Synergy       007-SBU  \
0       0  7.596576e+08   0  ...        0        0  1.133456e+08   
1       0  3.516687e+08   0  ...        0        0  0.000000e+00   
2       0  3.516687e+08   0  ...        0        0  0.000000e+00   
3       0  0.000000e+00   0  ...        0        0  0.000000e+00   
4       0  2.826361e+08   0  ...        0        0  0.000000e+00   

             TW  134-TPDS  135-INTNL OPER  138-CONT SOUR  140-NON MRV  \
0  1.448141e+09         0               0              0            0   
1  9.428187e+08         0               0              0            0   
2  9.428187e+08         0               0              0            0   
3  0.000000e+00         0               0              0            0   
4  9.366226e+08         0               0              0            0   

   141-SOURCING  139-CORP  
0             0         0  
1             0         0  
2             0         0  
3             0         0  
4             0         0  

[5 rows x 21 columns]

## Connecting to our local neo4j graph

In [1]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Print the URI to confirm it's loaded?
print(f"NEO4J_URI: {NEO4J_URI}")

# Create the Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Test the connection
try:
    with driver.session() as session:
        result = session.run("RETURN 1")
        if result.single()[0] == 1:
            print("Neo4j connection successful!")
        else:
            print("Neo4j connection failed!")
except Exception as e:
    print(f"Neo4j connection error: {e}")
finally:
    driver.close()


NEO4J_URI: bolt://localhost:7687
Neo4j connection successful!


## Adding Nodes and Relationships to our Neo4j KG

In [ ]:
with driver.session() as session:
    # Add GroupCompany nodes
    for company in group_companies:
        session.run(
            "MERGE (:GroupCompany {name: $name})",
            name=company
        )
    # Add InventoryType nodes
    for inv_type in inventory_types:
        session.run(
            "MERGE (:InventoryType {name: $name})",
            name=inv_type
        )


In [ ]:
## Adding Relationships between the nodes

def add_inventory_relationships(df, year):
    group_companies = list(df.columns[1:])  # Exclude the first column (InventoryType)
    for idx, row in df.iterrows():
        inventory_type = row[0]  # First column value (InventoryType)
        for company in group_companies:
            amount = row[company]
            # Skip missing or zero values (optional)
            if pd.isnull(amount) or amount == 0:
                continue
            with driver.session() as session:
                session.run(
                    """
                    MATCH (g:GroupCompany {name: $company})
                    MATCH (i:InventoryType {name: $inventory})
                    MERGE (g)-[r:HAS_INVENTORY {year: $year}]->(i)
                    SET r.amount = $amount
                    """,
                    company=company,
                    inventory=inventory_type,
                    year=year,
                    amount=float(amount)
                )


In [ ]:
add_inventory_relationships(df_fy23, "FY23")
add_inventory_relationships(df_fy24, "FY24")

In [ ]:
## Adding Inventories Node
with driver.session() as session:
    session.run(
        "MERGE (:Inventories {name: 'Inventories'})"
    )

In [ ]:
# Connecting all the other inventories types with relationship "inventory_type"

with driver.session() as session:
    session.run("""
        MATCH (inv:Inventories {name: 'Inventories'})
        MATCH (i:InventoryType)
        MERGE (i)-[:type_of_inventories]->(inv)
    """)


## Quering through our Neo4j Graph

In [ ]:
## Querying through our Knowledge Graph 
def run_cypher_query(cypher_query, parameters=None):
    with driver.session() as session:
        result = session.run(cypher_query, parameters or {})
        # Convert Neo4j result to list of dicts, then to DataFrame
        records = [record.data() for record in result]
    return pd.DataFrame(records)


In [ ]:
def run_cypher_scalar(cypher, params=None):
    with driver.session() as session:
        result = session.run(cypher, params or {})
        raw = [record.data() for record in result]
        return raw

In [ ]:
## Get All Relationships

cypher = """
MATCH (g:GroupCompany)-[r:HAS_INVENTORY]->(i:InventoryType)
RETURN g.name AS company, i.name AS inventory_type, r.year AS year, r.amount AS amount
ORDER BY company, year, inventory_type
"""
df_all = run_cypher_query(cypher)
print(df_all.head())



In [ ]:
## Get Inventory for a Specific Company and Year
cypher = """
MATCH (g:GroupCompany {name: $company})-[r:HAS_INVENTORY {year: $year}]->(i:InventoryType)
RETURN g.name AS company, i.name AS inventory_type, r.year AS year, r.amount AS amount
ORDER BY inventory_type
"""
params = {"company": "001-AS", "year": "FY24"}
df_company = run_cypher_query(cypher, params)
print(df_company)

In [ ]:
## Get Total Inventory by Company for FY24
cypher = """
MATCH (g:GroupCompany)-[r:HAS_INVENTORY {year:"FY24"}]->(i:InventoryType)
RETURN g.name AS company, SUM(r.amount) AS total_inventory
ORDER BY total_inventory DESC
"""
df_total = run_cypher_query(cypher)
print(df_total)


In [ ]:
## Calculate the difference between a inventory for a company for FY23 - FY24

def year_difference(company, inventory):
    cypher = """
    MATCH (g:GroupCompany {name: $company})-[r:HAS_INVENTORY]->(i:InventoryType {name: $inventory})
    WHERE r.year IN ['FY24', 'FY23']
    RETURN
      MAX(CASE WHEN r.year = 'FY24' THEN r.amount ELSE 0 END) AS FY24,
      MAX(CASE WHEN r.year = 'FY23' THEN r.amount ELSE 0 END) AS FY23,
      MAX(CASE WHEN r.year = 'FY24' THEN r.amount ELSE 0 END) - 
      MAX(CASE WHEN r.year = 'FY23' THEN r.amount ELSE 0 END) AS difference
    """
    params = {
        "company": company,
        "inventory": inventory
    }
    df = run_cypher_query(cypher, params)
    return df

# Example usage:
df_diff = year_difference(
    company="001-AS",
    inventory="Raw Material"
)
print(df_diff)


## Get the schema of the KG

In [7]:
def print_schema():
    with driver.session() as session:
        # Node labels
        labels = session.run("CALL db.labels()")
        print("Node Labels:")
        for record in labels:
            print(f" - {record[0]}")
        print("\n")

        # Relationship types
        rel_types = session.run("CALL db.relationshipTypes()")
        print("Relationship Types:")
        for record in rel_types:
            print(f" - {record[0]}")
        print("\n")

        # Property keys
        prop_keys = session.run("CALL db.propertyKeys()")
        print("Property Keys:")
        for record in prop_keys:
            print(f" - {record[0]}")
        print("\n")

        # Distinct patterns (from, rel, to)
        patterns = session.run("""
        MATCH (a)-[r]->(b)
        RETURN DISTINCT labels(a) AS from_labels, type(r) AS rel_type, labels(b) AS to_labels
        """)
        print("Relationship Patterns:")
        for record in patterns:
            print(f" - {record['from_labels']} -[{record['rel_type']}]-> {record['to_labels']}")
        print("\n")

In [8]:
print_schema()

Node Labels:
 - GroupCompany
 - InventoryType
 - Inventories


Relationship Types:
 - HAS_INVENTORY
 - type_of_inventories


Property Keys:
 - name
 - year
 - amount
 - id
 - data
 - nodes
 - relationships
 - style
 - visualisation


Relationship Patterns:
 - ['GroupCompany'] -[HAS_INVENTORY]-> ['InventoryType']
 - ['InventoryType'] -[type_of_inventories]-> ['Inventories']




C:\Users\bhavy\AppData\Local\Temp\ipykernel_436\1240237.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


## Closing the connection

In [ ]:
driver.close()

## More sample queries

In [ ]:
cypher1 = """
MATCH (g:GroupCompany {name: 'Powerol'})-[:HAS_INVENTORY]->(i:InventoryType)
RETURN count(DISTINCT i) AS inventoryTypeCount
"""
answer1 = run_cypher_scalar(cypher1)
print(f"Number of inventory types for Powerol: {answer1}")

In [ ]:
cypher2 = """
MATCH (g:GroupCompany {name: '008-SWARAJ'})-[h:HAS_INVENTORY {year: 'FY24'}]->(i:InventoryType)
RETURN sum(h.amount) AS totalAmount
"""
answer2 = run_cypher_scalar(cypher2)
print(f"Total inventory amount for 008-SWARAJ in 2023: {answer2}")

In [ ]:
cypher3 = """
MATCH (g:GroupCompany {name: '001-AS'})-[h:HAS_INVENTORY]->(i:InventoryType)
WHERE i.name = 'Stores and Spares' AND h.year IN ['FY23', 'FY24']
RETURN 
  sum(CASE WHEN h.year = 'FY24' THEN h.amount ELSE 0 END) -
  sum(CASE WHEN h.year = 'FY23' THEN h.amount ELSE 0 END) AS difference
"""
answer3 = run_cypher_scalar(cypher3)
print(f"Difference in 'Stores and Spares' for 001-AS (FY24 - FY23): {answer3}")

In [ ]:
cypher = """
MATCH (g:GroupCompany)-[h:HAS_INVENTORY]->(i:InventoryType)
WHERE h.year IN ['FY24', 'FY23']
WITH i.name AS inventory_type,
     sum(CASE WHEN h.year = 'FY24' THEN h.amount ELSE 0 END) AS fy24_total,
     sum(CASE WHEN h.year = 'FY23' THEN h.amount ELSE 0 END) AS fy23_total
RETURN inventory_type, fy24_total, fy23_total, (fy24_total - fy23_total) AS difference
ORDER BY abs(difference) DESC
LIMIT 1
"""
with driver.session() as session:
    record = session.run(cypher).single()
    if record:
        print(f"Inventory type with highest change: {record['inventory_type']}")
        print(f"  FY24 Total: {record['fy24_total']}")
        print(f"  FY23 Total: {record['fy23_total']}")
        print(f"  Difference: {record['difference']}")
    else:
        print("No data found.")


In [ ]:
cypher = """
MATCH (g:GroupCompany)-[h:HAS_INVENTORY]->(i:InventoryType)
WHERE h.year IN ['FY24', 'FY23']
WITH i.name AS inventory_type,
     sum(CASE WHEN h.year = 'FY24' THEN h.amount ELSE 0 END) AS fy24_total,
     sum(CASE WHEN h.year = 'FY23' THEN h.amount ELSE 0 END) AS fy23_total
RETURN
  inventory_type,
  fy24_total,
  fy23_total,
  (fy24_total - fy23_total) AS difference
ORDER BY abs(difference) DESC
LIMIT 1
"""
with driver.session() as session:
    record = session.run(cypher).single()
    if record:
        print(f"Inventory type with the highest difference between FY24 and FY23: ")
        print({record['inventory_type']})
        print(f"  FY24 Total: {record['fy24_total']}")
        print(f"  FY23 Total: {record['fy23_total']}")
        print(f"  Difference (FY24 - FY23): {record['difference']}")
    else:
        print("No data found.")


In [ ]:
cypher = """
MATCH (g:GroupCompany)-[h:HAS_INVENTORY {year: 'FY23'}]->(i:InventoryType)
RETURN g.name AS company_name, i.name AS inventory_type, h.amount AS amount
ORDER BY company_name, inventory_type
"""

with driver.session() as session:
    results = session.run(cypher)
    for record in results:
        print(
            f"Company: {record['company_name']}, "
            f"Inventory: {record['inventory_type']}, "
            f"Amount: {record['amount']}"
        )


In [ ]:
def total_inventory_for_company_and_year(company_name, year):
    cypher = """
    MATCH (g:GroupCompany {name: $company_name})-[h:HAS_INVENTORY {year: $year}]->(i:InventoryType)
    RETURN sum(h.amount) AS totalAmount
    """
    params = {
        "company_name": company_name,
        "year": year
    }
    with driver.session() as session:
        result = session.run(cypher, params)
        record = result.single()
        return record["totalAmount"] if record else None

# Example usage:s
total = total_inventory_for_company_and_year("002-HO", "FY23")
print(f"Total inventory for 002-HO in FY24: {total}")


In [ ]:
cypher = """
MATCH (g:GroupCompany)-[h:HAS_INVENTORY {year: 'FY24'}]->(i:InventoryType {name: 'Raw Material'})
RETURN g.name AS company_name, h.amount AS raw_material_amount
ORDER BY raw_material_amount DESC
LIMIT 1
"""

with driver.session() as session:
    result = session.run(cypher)
    record = result.single()
    if record:
        print(f"Group company with highest Raw Material in FY24: {record['company_name']} ({record['raw_material_amount']})")
    else:
        print("No data found.")
